In [1]:
import sys
from pathlib import Path
from threading import Thread
from subprocess import Popen, PIPE
from collections import defaultdict

import numpy as np
import pandas as pd

In [2]:
!ls ../out

2022-08-14T21:24:49.475669
2022-08-14T22:24:50.595949_6397.73_sa_5_0.5
2022-08-14T22:27:21.957662_6670.33_sa_double_tl
2022-08-14T22:41:23.433876_7152.92_sa_5x_tl
2022-08-15T05:32:03.216882_6276.69
2022-08-15T05:36:22.615730_6391.41
2022-08-15T05:38:34.041271_6332.39
2022-08-15T05:43:06.292641_6409.07
2022-08-15T06:05:37.168062_6404.80


In [3]:
scores = {}

def read_stream(name, in_file, out_file):
    for line in in_file:
        #print(f"[{name}] {line.strip()}", file=out_file)
        try:
            scores[name] = int(line.strip().split()[-1])
        except:
            pass

def run(cmd, name):
    proc = Popen(cmd, stdout=PIPE, stderr=PIPE, universal_newlines=True, shell=True)
    stdout_thread = Thread(target=read_stream, args=(name, proc.stdout, sys.stdout))
    stderr_thread = Thread(target=read_stream, args=(name, proc.stderr, sys.stderr))
    stdout_thread.start()
    stderr_thread.start()
    proc.wait()
    return proc

N = 100
for out_dir in Path("../out").iterdir():
    for i in range(N):
        out_file = out_dir / f"{i:04d}.txt"
        cmd = f"../tools/target/release/vis ../tools/in/{i:04d}.txt {out_file}"
        run(cmd, (out_dir, i))

In [4]:
dict_out_dir_to_scores = defaultdict(lambda: [0] * N)
for (out_dir, i), score in scores.items():
    dict_out_dir_to_scores[out_dir][i] = score

In [5]:
df = pd.DataFrame(dict_out_dir_to_scores)
df.corr()

,../out/2022-08-15T05:32:03.216882_6276.69,../out/2022-08-14T22:27:21.957662_6670.33_sa_double_tl,../out/2022-08-15T05:38:34.041271_6332.39,../out/2022-08-15T05:43:06.292641_6409.07,../out/2022-08-15T06:05:37.168062_6404.80,../out/2022-08-15T05:36:22.615730_6391.41,../out/2022-08-14T22:41:23.433876_7152.92_sa_5x_tl,../out/2022-08-14T21:24:49.475669,../out/2022-08-14T22:24:50.595949_6397.73_sa_5_0.5
../out/2022-08-15T05:32:03.216882_6276.69,1.000000,0.851705,0.869650,0.859589,0.796054,0.880551,0.885151,0.496172,0.871190
../out/2022-08-14T22:27:21.957662_6670.33_sa_double_tl,0.851705,1.000000,0.878589,0.916477,0.835372,0.870808,0.874795,0.514055,0.929135
../out/2022-08-15T05:38:34.041271_6332.39,0.869650,0.878589,1.000000,0.894722,0.811186,0.886025,0.888408,0.491465,0.874347
../out/2022-08-15T05:43:06.292641_6409.07,0.859589,0.916477,0.894722,1.000000,0.837025,0.876540,0.881056,0.591354,0.952038
../out/2022-08-15T06:05:37.168062_6404.80,0.796054,0.835372,0.811186,0.837025,1.000000,0.806580,0.830635,0.582321,0.878082
../out/2022-08-15T05:36:22.615730_6391.41,0.880551,0.870808,0.886025,0.876540,0.806580,1.000000,0.885192,0.459641,0.863876
../out/2022-08-14T22:41:23.433876_7152.92_sa_5x_tl,0.885151,0.874795,0.888408,0.881056,0.830635,0.885192,1.000000,0.458834,0.873963
../out/2022-08-14T21:24:49.475669,0.496172,0.514055,0.491465,0.591354,0.582321,0.459641,0.458834,1.000000,0.589278
../out/2022-08-14T22:24:50.595949_6397.73_sa_5_0.5,0.871190,0.929135,0.874347,0.952038,0.878082,0.863876,0.873963,0.589278,1.000000


In [6]:
df.values.argmax(1)

array([5, 6, 6, 1, 6, 4, 6, 6, 4, 4, 2, 6, 6, 6, 6, 5, 6, 6, 6, 3, 6, 3,
       6, 6, 6, 8, 0, 1, 1, 1, 0, 6, 6, 5, 6, 6, 4, 6, 1, 6, 3, 6, 6, 6,
       6, 4, 6, 6, 6, 4, 6, 6, 6, 6, 6, 6, 6, 4, 1, 6, 1, 0, 6, 6, 1, 6,
       4, 5, 6, 6, 6, 6, 6, 6, 1, 6, 6, 6, 4, 1, 6, 2, 6, 6, 6, 1, 6, 4,
       6, 3, 1, 1, 1, 4, 3, 6, 6, 0, 0, 6])

In [7]:
np.bincount(df.values.argmax(1))

array([ 5, 14,  2,  5, 11,  4, 58,  0,  1])

In [8]:
df.columns

Index([             ../out/2022-08-15T05:32:03.216882_6276.69,
       ../out/2022-08-14T22:27:21.957662_6670.33_sa_double_tl,
                    ../out/2022-08-15T05:38:34.041271_6332.39,
                    ../out/2022-08-15T05:43:06.292641_6409.07,
                    ../out/2022-08-15T06:05:37.168062_6404.80,
                    ../out/2022-08-15T05:36:22.615730_6391.41,
           ../out/2022-08-14T22:41:23.433876_7152.92_sa_5x_tl,
                            ../out/2022-08-14T21:24:49.475669,
           ../out/2022-08-14T22:24:50.595949_6397.73_sa_5_0.5],
      dtype='object')